In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style("dark")

In [2]:
to_drop = ['Unnamed: 0', 'title', 'song_name', 'analysis_url', 'uri', 'track_href', 'type']
df_original = pd.read_csv('genres_v2.csv')
df = df_original.drop(to_drop, axis=1)
df.head()

C:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,genre
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,2Vc6NJ9PW9gD9q343XFRKx,124539,4,Dark Trap
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,7pgJBLVz5VmnL7uGHmRj6p,224427,4,Dark Trap
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,0vSWgAlfpye0WCGeNmuNhy,98821,4,Dark Trap
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,0VSXnJqQkwuH2ei1nOQ1nu,123661,3,Dark Trap
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,4jCeguq9rMTlbMmPHuO7S3,123298,4,Dark Trap


In [13]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['genre_encoded'] = encoder.fit_transform(df['genre'])
df.corr()['genre_encoded']

danceability       -0.052687
energy              0.471327
key                 0.027398
loudness            0.160771
mode               -0.019531
speechiness        -0.144596
acousticness       -0.356286
instrumentalness    0.414434
liveness            0.107690
valence            -0.170698
tempo              -0.231731
duration_ms         0.412508
time_signature      0.019943
genre_encoded       1.000000
Name: genre_encoded, dtype: float64

In [23]:
encoder.classes_

array(['Dark Trap', 'Emo', 'Hiphop', 'Pop', 'Rap', 'RnB', 'Trap Metal',
       'Underground Rap', 'dnb', 'hardstyle', 'psytrance', 'techhouse',
       'techno', 'trance', 'trap'], dtype=object)

In [24]:
encoder.transform(['Dark Trap', 'Emo', 'Hiphop', 'Pop', 'Rap', 'RnB', 'Trap Metal',
       'Underground Rap', 'dnb', 'hardstyle', 'psytrance', 'techhouse',
       'techno', 'trance', 'trap'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [14]:
features = df.drop(['mode', 'time_signature', 'key', 'danceability', 'genre', 'genre_encoded', 'id'],axis=1)
features.head()

,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.814,-7.364,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,124539
1,0.493,-7.230,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,224427
2,0.893,-4.783,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,98821
3,0.781,-4.710,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,123661
4,0.624,-7.668,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,123298


In [15]:
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
y = df['genre_encoded']
features = MinMaxScaler().fit_transform(features)
oversample = SMOTE()
features, labels = oversample.fit_resample(features, y)

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True)

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier()

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [18]:
n_scores.mean()

0.7541560283687941

In [19]:
from sklearn.metrics import accuracy_score
model = BaggingClassifier()
model.fit(x_train, y_train)
pred = model.predict(x_test)
accuracy_score(y_test, pred)

0.771177304964539

In [25]:
import pickle
pickle.dump(encoder, open("resources/lable_encoder.pkl", "wb"))
pickle.dump(model, open("resources/classifier.pkl", "wb"))